<a href="https://colab.research.google.com/github/stemlock/w266_final_project/blob/master/NF_Base_Model_Colab_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment Setup

In [ ]:
from google.colab import drive
from google.colab import auth
auth.authenticate_user()

drive.mount('/content/drive')
CWD = '/content/drive/My Drive/W266 Final Project/Code'

%cd $CWD

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1TVsdzndbQ4mnh6_mVPmR4xoE28WX7oHG/W266 Final Project/Code


In [ ]:
! pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-cqe88pli
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-cqe88pli
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 5.2 MB/s 
     |████████████████████████████████| 61 kB 447 kB/s 
     |████████████████████████████████| 596 kB 38.4 MB/s 
     |████████████████████████████████| 3.3 MB 44.5 MB/s 
  Created wheel for transformers: filename=transformers-4.13.0.dev0-py3-none-any.whl size=3254938 sha256=ba29286bc1fe2153f624e3560e692a1fe2235f548b3222f4d214f19d58575127
  Stored in directory: /tmp/pip-ephem-wheel-cache-vbkjs8x4/wheels/90/a5/44/6bcd83827c8a60628c5ad602f429cd5076bcce5f2a90054947
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfu

In [ ]:
# Imports
import os

import pandas as pd
import numpy as np
from csv import writer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
import transformers
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

In [ ]:
print("Tensorflow version:", tf.__version__)
print("Transformers version:", transformers.__version__)

Tensorflow version: 2.7.0
Transformers version: 4.13.0.dev0


In [ ]:
# Set random seed
seed = 42

## Load Data

In [ ]:
# Read in processed data (Rows with NA in the neutral_review_text had no tokens replaced)
df_train = pd.read_csv('data/model_train_v2.csv')
df_test = pd.read_csv('data/model_test_v2.csv')
df_train.head()

,review_id,review_score,review_text,neutral_review_text,neutral_sub_count,female_review_text,male_sub_count,male_review_text,female_sub_count,label
0,3631,1,I first watched this show hoping for a few lau...,i first watched this show hoping for a few lau...,4,i first watched this show hoping for a few lau...,1,i first watched this show hoping for a few lau...,4,0
1,7264,1,Just another film that exploits gratuitous fro...,just another film that exploits gratuitous fro...,8,just another film that exploits gratuitous fro...,6,just another film that exploits gratuitous fro...,4,0
2,6649,1,"OK, anyone who could honestly say that this mo...","ok, anyone who could honestly [UNK] that this ...",6,"ok, anyone who could honestly mary that this m...",4,"ok, anyone who could honestly say that this mo...",2,0
3,6443,1,I was one of the few non-liberals who showed u...,i was one of the few non-liberals who showed u...,11,i was one of the few non-liberals who showed u...,11,i was one of the few non-liberals who showed u...,0,0
4,4359,1,Suburban kids meet the forest. Killjoy is bett...,suburban kids meet the [UNK]. killjoy is bette...,9,suburban kids meet the mary. killjoy is better...,7,suburban kids meet the forest. killjoy is bett...,2,0


## Baseline Metrics

### Majority tokens average

In [ ]:
# Extract the male vs female majority texts
df_female_majority = df_test[df_test['male_sub_count'] > df_test['female_sub_count']]
df_male_majority = df_test[df_test['male_sub_count'] < df_test['female_sub_count']]

In [ ]:
# Average sentiment for female majority texts
print("Average female review binary sentiment:", df_female_majority['label'].mean())

Average female review binary sentiment: 0.49158031088082904


In [ ]:
# Average review score for female majority texts
print("Average female review score 1-10:", df_female_majority['review_score'].mean())

Average female review score 1-10: 5.4220207253886015


In [ ]:
# Average sentiment for male majority texts
print("Average male review binary sentiment:", df_male_majority['label'].mean())

Average male review binary sentiment: 0.5467349551856594


In [ ]:
# Average review score for male majority texts
print("Average male review score 1-10:", df_male_majority['review_score'].mean())

Average male review score 1-10: 5.87708066581306


In [ ]:
# Distribution of scores across review scores for female majority texts
df_female_majority['review_score'].value_counts()/len(df_female_majority)

1     0.209067
10    0.187435
8     0.114896
4     0.106736
3     0.098057
7     0.097021
2     0.094560
9     0.092228
Name: review_score, dtype: float64

In [ ]:
# Distribution of scores across review scores for male majority texts
df_male_majority['review_score'].value_counts()/len(df_male_majority)

10    0.215749
1     0.156210
8     0.135083
4     0.111396
3     0.102433
7     0.099872
9     0.096031
2     0.083227
Name: review_score, dtype: float64

### Proportional weighted average

In [ ]:
# Extract the proportion of male vs female tokens per review
male_proportion = (df_test['female_sub_count']/df_test['neutral_sub_count'])
female_proportion = (df_test['male_sub_count']/df_test['neutral_sub_count'])

In [ ]:
# Weighted average sentiment for female tokens
(df_test['label']*female_proportion).sum()/female_proportion.sum()

0.4942995300910007

In [ ]:
# Weighted average review scores for female tokens
(df_test['review_score']*female_proportion).sum()/female_proportion.sum()

5.442629024859665

In [ ]:
# Weighted average sentiment for male tokens
(df_test['label']*male_proportion).sum()/male_proportion.sum()

0.5225755644647283

In [ ]:
# Weighted average review scores for male tokens
(df_test['review_score']*male_proportion).sum()/male_proportion.sum()

5.689589030131117

## Data Transformations

#### Function Approach (Not working)

In [ ]:
# ## TO DO: Try to fix this function. For some reason, this causes the model.fit() to fail
# def encode_datasets(X_train, y_train, X_test, y_test, tokenizer, split_size=0.5, seed=42):

#   '''
#   Takes in train and test data and encodes them into train, dev, and test
#   TF datasets using the provided tokenizer.
#   '''

#   # Split test set into dev and test
#   X_dev, X_test, y_dev, y_test = train_test_split(X_test, y_test, test_size=split_size, random_state=seed)

#   # Apply tokenizer to each dataset
#   train_encodings = tokenizer(X_train, truncation=True, padding=True)
#   dev_encodings = tokenizer(X_dev, truncation=True, padding=True)
#   test_encodings = tokenizer(X_test, truncation=True, padding=True)

#   # Turn encodings into datasets for easy batching
#   train_dataset = tf.data.Dataset.from_tensor_slices((
#       dict(train_encodings),
#       y_train
#   ))
#   dev_dataset = tf.data.Dataset.from_tensor_slices((
#       dict(dev_encodings),
#       y_dev
#   ))
#   test_dataset = tf.data.Dataset.from_tensor_slices((
#       dict(test_encodings),
#       y_test
#   ))

#   return train_dataset, dev_dataset, test_dataset

In [ ]:
# # Load data
# train_texts = df_train['review_text'].values.tolist()
# n_train_texts = df_train['neutral_review_text'].values.tolist()
# f_train_texts = df_train['female_review_text'].values.tolist()
# m_train_texts = df_train['male_review_text'].values.tolist()
# train_labels = df_train['label'].values.tolist()

# test_texts = df_test['review_text'].values.tolist()
# n_test_texts = df_test['neutral_review_text'].values.tolist()
# f_test_texts = df_test['female_review_text'].values.tolist()
# m_test_texts = df_test['male_review_text'].values.tolist()
# test_labels = df_test['label'].values.tolist()

In [ ]:
# # Specify tokenizer and encode each dataset
# tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# train_dataset, dev_dataset, test_dataset = encode_datasets(train_texts, train_labels, 
#                                                            test_texts, test_labels,
#                                                            tokenizer)
# n_train_dataset, n_dev_dataset, n_test_dataset = encode_datasets(n_train_texts, train_labels, 
#                                                                  n_test_texts, test_labels,
#                                                                  tokenizer)

### Split data

In [ ]:
# Load data
train_texts = df_train['review_text'].values.tolist()
n_train_texts = df_train['neutral_review_text'].values.tolist()
f_train_texts = df_train['female_review_text'].values.tolist()
m_train_texts = df_train['male_review_text'].values.tolist()
train_labels = df_train['label'].values.tolist()

test_texts = df_test['review_text'].values.tolist()
n_test_texts = df_test['neutral_review_text'].values.tolist()
f_test_texts = df_test['female_review_text'].values.tolist()
m_test_texts = df_test['male_review_text'].values.tolist()
test_labels = df_test['label'].values.tolist()

In [ ]:
# Create dev set from portion of test set using split
dev_texts, test_texts, _, _ = train_test_split(test_texts, test_labels, test_size=.5, random_state=seed)
n_dev_texts, n_test_texts, _, _ = train_test_split(n_test_texts, test_labels, test_size=.5, random_state=seed)
f_dev_texts, f_test_texts, _, _ = train_test_split(f_test_texts, test_labels, test_size=.5, random_state=seed)
m_dev_texts, m_test_texts, dev_labels, test_labels = train_test_split(m_test_texts, test_labels, 
                                                                        test_size=.5, random_state=seed)

### Tokenize

In [ ]:
# Specify tokenizer and batch encode original datasets
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='tf')
dev_encodings = tokenizer(dev_texts, truncation=True, padding=True, return_tensors='tf')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='tf')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
# Batch encode the neutral datasets
n_train_encodings = tokenizer(n_train_texts, truncation=True, padding=True, return_tensors='tf')

In [ ]:
# Batch encode the male and female datasets
m_test_encodings = tokenizer(m_test_texts, truncation=True, padding=True, return_tensors='tf')
f_test_encodings = tokenizer(f_test_texts, truncation=True, padding=True, return_tensors='tf')

### Create TF.Datasets

In [ ]:
# Change labels list into tf.Tensors

tf_train_labels = tf.convert_to_tensor(train_labels)
tf_dev_labels = tf.convert_to_tensor(dev_labels)
tf_test_labels = tf.convert_to_tensor(test_labels)

In [ ]:
# Turn original encodings into datasets for easy batching

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    tf_train_labels
)).shuffle(10000, seed=seed).batch(16)

dev_dataset = tf.data.Dataset.from_tensor_slices((
    dict(dev_encodings),
    tf_dev_labels
)).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    tf_test_labels
)).batch(16)

In [ ]:
# Turn neutral encodings into datasets for easy batching

n_train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(n_train_encodings),
    tf_train_labels
)).shuffle(10000, seed=seed).batch(16)


In [ ]:
# Turn male and female encodings into datasets for easy batching

m_test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(m_test_encodings),
    tf_test_labels
)).batch(16)


f_test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(f_test_encodings),
    tf_test_labels
)).batch(16)

## Model Pipeline

### Initialize TF strategy (TPU preferred)

In [ ]:
# Initialize the TPU devices
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.40.201.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.40.201.210:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


### Baseline Model Architecture

In [ ]:
# Starter function to create the model (we can improve on this when we start using more complex models)
def create_model():

  return TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

### Baseline Model v1 (5 epochs)

#### Train model

In [ ]:
# Create the model within each device scope
base_models = []
base_histories = []

num_epochs = 5
learn_rate = 1e-5

with open('./saved_results/results.csv', 'a+', newline='') as write_obj:
    csv_writer = writer(write_obj)
    csv_writer.writerow(['label', 'num_epochs', 'learn_rate', 'loss', 'accuracy', 'avg_m_pos_sent', 'avg_f_pos_sent', 'sent_diff'])

for i in range(4): 
  for label, train, dev in [('original', train_dataset, dev_dataset), ('neutral', n_train_dataset, dev_dataset)]:
    with strategy.scope():
      model = create_model()
      optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate)
      model.compile(optimizer=optimizer, 
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                    metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
      
    print(model.summary())

    history = model.fit(train, validation_data=dev, epochs=num_epochs, shuffle=False)
    base_histories.append(history)
    base_models.append(model)

    eval = model.evaluate(x=test_dataset)
    loss = eval[0]
    accuracy = eval[1]

    orig_m_logit_preds = model.predict(x=m_test_dataset).logits
    orig_f_logit_preds = model.predict(x=f_test_dataset).logits   

    m_pred_probs = tf.math.softmax(orig_m_logit_preds, axis=-1)
    f_pred_probs = tf.math.softmax(orig_f_logit_preds, axis=-1)

    model_stats = [label, num_epochs, learn_rate, loss, accuracy, np.mean(m_pred_probs[:,1]), np.mean(f_pred_probs[:,1]), (np.mean(m_pred_probs[:,1])-np.mean(f_pred_probs[:,1]))]

    with open('./saved_results/results.csv', 'a+', newline='') as write_obj:
        csv_writer = writer(write_obj)
        csv_writer.writerow(model_stats)

  print('~~~~~~Finished iteration ' + str(i) + '~~~~~~~')

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond/Identity_16:0' shape=(None,) dtype=int32>]


Instructions for updating:
use `experimental_local_results` instead.


...

2462/2462 [==============================] - ETA: 0s - loss: 0.1766 - accuracy: 0.9313

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond/Identity_16:0' shape=(None,) dtype=int32>]


2462/2462 [==============================] - 231s 73ms/step - loss: 0.1766 - accuracy: 0.9313 - val_loss: 0.2377 - val_accuracy: 0.9090
Epoch 2/5
2462/2462 [==============================] - 172s 70ms/step - loss: 0.1174 - accuracy: 0.9555 - val_loss: 0.3071 - val_accuracy: 0.8962
Epoch 3/5
2462/2462 [==============================] - 173s 70ms/step - loss: 0.0859 - accuracy: 0.9684 - val_loss: 0.2672 - val_accuracy: 0.9076
Epoch 4/5
2462/2462 [==============================] - 172s 70ms/step - loss: 0.0599 - accuracy: 0.9798 - val_loss: 0.3071 - val_accuracy: 0.9015
Epoch 5/5
308/308 [==============================] - 10s 29ms/step - loss: 0.2401 - accuracy: 0.9188


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond/Identity_16:0' shape=(None,) dtype=int32>]


~~~~~~Finished iteration 3~~~~~~~
